In [1]:
# formulary_tier_landscape.py
import os
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

# ---------------------------
# 1) DB connection (your snippet)
# ---------------------------
load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)

In [2]:
# helper: run SQL and return pandas DataFrame
def run_sql(sql: str) -> pd.DataFrame:
    """Execute SQL and return DataFrame."""
    return pd.read_sql_query(sql, conn)

In [3]:
# ---------------------------
# 2) Basic config & assumptions
# ---------------------------
# tiers considered favorable (change if needed)
favorable_tiers = [1, 2]

# folder to save results
OUT_DIR = "formulary_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

In [4]:
# ---------------------------
# 3) Tier distribution by formulary_id and contract_year
#    - counts and distinct RXCUI per tier
# ---------------------------
sql_formulary_year_tier = """
WITH base AS (
  SELECT formulary_id,
         contract_year,
         COALESCE(tier_level_value, -1) AS tier_level_value,
         rxcui
  FROM basic_drugs_formulary
)
SELECT
  formulary_id,
  contract_year,
  tier_level_value,
  COUNT(*) AS rows_count,
  COUNT(DISTINCT rxcui) AS distinct_rxcui
FROM base
GROUP BY formulary_id, contract_year, tier_level_value
ORDER BY formulary_id, contract_year, tier_level_value;
"""

print("Querying tier distribution by formulary_id & contract_year ...")
df_formulary = run_sql(sql_formulary_year_tier)

# compute percentages of distinct_rxcui within each (formulary_id, contract_year)
df_formulary['distinct_rxcui'] = df_formulary['distinct_rxcui'].astype(int)
df_formulary['pct_of_formulary'] = (
    df_formulary
    .groupby(['formulary_id', 'contract_year'])['distinct_rxcui']
    .apply(lambda x: 100.0 * x / x.sum())
    .reset_index(level=[0,1], drop=True)
)

# save
df_formulary.to_csv(os.path.join(OUT_DIR, "tier_by_formulary_contractyear.csv"), index=False)
print("Saved: tier_by_formulary_contractyear.csv (rows: {})".format(len(df_formulary)))

# Pivot example: each tier as column (distinct_rxcui)
pivot_formulary = df_formulary.pivot_table(
    index=['formulary_id','contract_year'],
    columns='tier_level_value',
    values='distinct_rxcui',
    fill_value=0
).reset_index()
pivot_formulary.to_csv(os.path.join(OUT_DIR, "pivot_formulary_contractyear_by_tier.csv"), index=False)

Querying tier distribution by formulary_id & contract_year ...


C:\Users\amsub\AppData\Local\Temp\ipykernel_44120\969165552.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, conn)


Saved: tier_by_formulary_contractyear.csv (rows: 1576)


In [ ]:

# # ---------------------------
# # 4) Tier distribution by geography (state, county) and region codes
# #    Join: basic_drugs_formulary -> plan_info (on formulary_id)
# #          then plan_info -> geographic_locator (on county_code)
# #    We compute: number of distinct plans with drug at given tier per county/state
# # ---------------------------
# sql_geo_tier = """
# SELECT
#   pi.ma_region_code,
#   pi.pdp_region_code,
#   pi.state,
#   pi.county_code,
#   gl.statename AS geo_statename,
#   gl.county AS geo_county_name,
#   COALESCE(bd.tier_level_value, -1) AS tier_level_value,
#   COUNT(DISTINCT pi.plan_id)      AS n_plans,
#   COUNT(DISTINCT bd.rxcui)        AS distinct_rxcui
# FROM basic_drugs_formulary bd
# JOIN plan_info pi ON bd.formulary_id = pi.formulary_id
# LEFT JOIN geographic_locator gl ON pi.county_code = gl.county_code
# GROUP BY
#   pi.ma_region_code, pi.pdp_region_code, pi.state, pi.county_code, gl.statename, gl.county, bd.tier_level_value
# ORDER BY pi.state NULLS LAST, pi.county_code NULLS LAST, tier_level_value
# LIMIT 5000;
# """

# cur = conn.cursor()
# cur.execute("SHOW work_mem;")
# current_value = cur.fetchone()[0]
# print("Current work_mem:", current_value)
# # Set work_mem for this session
# cur.execute("SET work_mem = '1024MB';")
# cur.execute("SHOW work_mem;")
# current_value = cur.fetchone()[0]
# print("Current work_mem:", current_value)

# print("Querying tier distribution by geography ...")
# df_geo = run_sql(sql_geo_tier)

# cur = conn.cursor()
# cur.execute("SHOW work_mem;")
# current_value = cur.fetchone()[0]
# print("Current work_mem:", current_value)
# # Set work_mem for this session
# cur.execute("SET work_mem = '4MB';")
# cur.execute("SHOW work_mem;")
# current_value = cur.fetchone()[0]
# print("Current work_mem:", current_value)

# # percentages of plans by tier within each (state, county)
# df_geo['n_plans'] = df_geo['n_plans'].astype(int)
# df_geo['distinct_rxcui'] = df_geo['distinct_rxcui'].astype(int)
# df_geo['pct_plans_in_tier'] = (
#     df_geo
#     .groupby(['state','county_code'])['n_plans']
#     .apply(lambda x: 100.0 * x / x.sum())   
#     .reset_index(level=[0,1], drop=True)
# )

# df_geo.to_csv(os.path.join(OUT_DIR, "tier_by_geo_state_county.csv"), index=False)
# print("Saved: tier_by_geo_state_county.csv (rows: {})".format(len(df_geo)))

Current work_mem: 4MB
Current work_mem: 1GB
Querying tier distribution by geography ...


C:\Users\amsub\AppData\Local\Temp\ipykernel_40412\969165552.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, conn)


Current work_mem: 1GB
Current work_mem: 4MB
Saved: tier_by_geo_state_county.csv (rows: 5000)


In [ ]:
# conn.autocommit = True
cur = conn.cursor()

sql_split_query = """
WITH bdf_agg AS (
  SELECT formulary_id, tier_level_value,
         COUNT(DISTINCT rxcui) AS distinct_rxcui_count
  FROM basic_drugs_formulary
  GROUP BY formulary_id, tier_level_value
),
plan_geo_agg AS (
  SELECT pi.ma_region_code, pi.pdp_region_code, pi.state, pi.county_code,
         gl.statename AS geo_statename, gl.county AS geo_county_name,
         COALESCE(bdf_agg.tier_level_value, -1) AS tier_level_value,
         COUNT(DISTINCT pi.plan_id) AS n_plans,
         SUM(bdf_agg.distinct_rxcui_count) AS distinct_rxcui
  FROM plan_info pi
  JOIN bdf_agg ON pi.formulary_id = bdf_agg.formulary_id
  LEFT JOIN geographic_locator gl ON pi.county_code = gl.county_code
  GROUP BY pi.ma_region_code, pi.pdp_region_code, pi.state, pi.county_code,
           gl.statename, gl.county, bdf_agg.tier_level_value
)
SELECT *,
       n_plans * 100.0 / SUM(n_plans) OVER (PARTITION BY state, county_code) AS pct_plans_in_tier
FROM plan_geo_agg
ORDER BY state NULLS LAST, county_code NULLS LAST, tier_level_value
"""
# LIMIT 5000;


cur = conn.cursor()
cur.execute("SHOW work_mem;")
current_value = cur.fetchone()[0]
print("Current work_mem:", current_value)
# Set work_mem for this session
cur.execute("SET work_mem = '1024MB';")
cur.execute("SHOW work_mem;")
current_value = cur.fetchone()[0]
print("Current work_mem:", current_value)

print("Running the split aggregation query...")
cur.execute(sql_split_query)
rows = cur.fetchall()
# Optional: get column names
colnames = [desc[0] for desc in cur.description]

cur = conn.cursor()
cur.execute("SHOW work_mem;")
current_value = cur.fetchone()[0]
print("Current work_mem:", current_value)
# Set work_mem for this session
cur.execute("SET work_mem = '4MB';")
cur.execute("SHOW work_mem;")
current_value = cur.fetchone()[0]
print("Current work_mem:", current_value)




# Convert to list of dicts or pandas DataFrame as needed
import pandas as pd
df = pd.DataFrame(rows, columns=colnames)

print(f"Query returned {len(df)} rows")
print(df.head())


Current work_mem: 4MB
Current work_mem: 1GB
Running the split aggregation query...
Current work_mem: 1GB
Current work_mem: 4MB
Query returned 18753 rows
  ma_region_code pdp_region_code state county_code geo_statename  \
0             07                                            None   
1                             23                            None   
2             15                                            None   
3                             11                            None   
4                             22                            None   

  geo_county_name  tier_level_value  n_plans distinct_rxcui  \
0            None                 1        1            457   
1            None                 1       15           3779   
2            None                 1        4           1925   
3            None                 1       14           3611   
4            None                 1       15           3779   

        pct_plans_in_tier  
0  0.03949447077409162717  
1  

In [14]:
df.to_csv(os.path.join(OUT_DIR, "tier_by_geo_state_county.csv"), index=False)
print("Saved: tier_by_geo_state_county.csv (rows: {})".format(len(df)))

Saved: tier_by_geo_state_county.csv (rows: 18753)


In [13]:
# ---------------------------
# 5) Aggregated summary by state (good for quick prioritization)
# ---------------------------
state_summary = (
    df
    .groupby(['state', 'tier_level_value'])
    .agg(
        total_plans_in_tier = ('n_plans', 'sum'),
        distinct_rxcui = ('distinct_rxcui', 'sum')
    )
    .reset_index()
)
# compute percent of plans by tier per state
state_summary['pct_plans_in_tier'] = (
    state_summary.groupby('state')['total_plans_in_tier']
    .transform(lambda x: 100.0 * x / x.sum())
)
state_summary.to_csv(os.path.join(OUT_DIR, "state_tier_summary.csv"), index=False)
print("Saved: state_tier_summary.csv (rows: {})".format(len(state_summary)))

Saved: state_tier_summary.csv (rows: 308)


In [15]:
# ---------------------------
# 6) Quick charts: stacked bar of tier distribution over contract_year and by state
# ---------------------------
# 6a) Contract year vs tier (global)
print("Preparing contract_year vs tier plot ...")
sql_year_tier = """
SELECT contract_year, COALESCE(tier_level_value, -1) AS tier_level_value,
       COUNT(DISTINCT rxcui) AS distinct_rxcui
FROM basic_drugs_formulary
GROUP BY contract_year, COALESCE(tier_level_value, -1)
ORDER BY contract_year, tier_level_value;
"""
df_year_tier = run_sql(sql_year_tier)
df_year_tier['contract_year'] = df_year_tier['contract_year'].astype(int)
pivot_year = df_year_tier.pivot(index='contract_year', columns='tier_level_value', values='distinct_rxcui').fillna(0)

# plot stacked bar (contract_year)
ax = pivot_year.plot(kind='bar', stacked=True, figsize=(12,6))
ax.set_title("Distinct RXCUI counts by contract_year and tier_level")
ax.set_ylabel("Distinct RXCUI")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "contract_year_tier_stacked.png"))
plt.close()
print("Saved: contract_year_tier_stacked.png")

Preparing contract_year vs tier plot ...


C:\Users\amsub\AppData\Local\Temp\ipykernel_44120\969165552.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, conn)


Saved: contract_year_tier_stacked.png


In [16]:
# 6b) State-level stacked bar (top 12 states by total plans)
print("Preparing top states tier distribution plot ...")
state_total = state_summary.groupby('state')['total_plans_in_tier'].sum().reset_index().rename(columns={'total_plans_in_tier':'total_plans'})
top_states = state_total.sort_values('total_plans', ascending=False).head(12)['state'].tolist()
state_top_df = state_summary[state_summary['state'].isin(top_states)]
pivot_state = state_top_df.pivot(index='state', columns='tier_level_value', values='total_plans_in_tier').fillna(0)
ax = pivot_state.plot(kind='bar', stacked=True, figsize=(12,6))
ax.set_title("Top 12 States: Distribution of Plans by Tier Level")
ax.set_ylabel("Number of Plans (in tier)")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "top_states_tier_stacked.png"))
plt.close()
print("Saved: top_states_tier_stacked.png")

Preparing top states tier distribution plot ...
Saved: top_states_tier_stacked.png


In [17]:
# ---------------------------
# 7) Helpful ranking outputs
#    - Top 30 formulary_ids with highest percent of drugs in unfavourable tiers
# ---------------------------
print("Computing top formularies with high share of unfavourable tiers...")
# define unfavourable tiers (example: >=3)
unfavorable_mask = df_formulary['tier_level_value'] >= 3
unfav_df = (
    df_formulary[unfavorable_mask]
    .groupby(['formulary_id','contract_year'])
    .agg(unfav_distinct_rxcui=('distinct_rxcui','sum'))
    .reset_index()
)
total_df = (
    df_formulary
    .groupby(['formulary_id','contract_year'])
    .agg(total_distinct_rxcui=('distinct_rxcui','sum'))
    .reset_index()
)
merged = fav = pd.merge(total_df, unfav_df, on=['formulary_id','contract_year'], how='left').fillna(0)
merged['pct_unfavorable'] = merged['unfav_distinct_rxcui'] / merged['total_distinct_rxcui'] * 100.0
top_problematic = merged.sort_values('pct_unfavorable', ascending=False).head(30)
top_problematic.to_csv(os.path.join(OUT_DIR,"top_30_formularies_pct_unfavorable.csv"), index=False)
print("Saved: top_30_formularies_pct_unfavorable.csv")

Computing top formularies with high share of unfavourable tiers...
Saved: top_30_formularies_pct_unfavorable.csv


In [18]:
# ---------------------------
# 8) Example outputs for quick manual review
# ---------------------------
# Save a sample of county-level rows where pct_plans_in_tier > 50 for a high tier (>=3)
df_geo_high = df[(df['tier_level_value'] >= 3) & (df['pct_plans_in_tier'] > 50)]
df_geo_high.sort_values(['state','pct_plans_in_tier'], ascending=[True, False]).to_csv(
    os.path.join(OUT_DIR, "counties_high_tier_majority.csv"), index=False
)
print("Saved: counties_high_tier_majority.csv (rows: {})".format(len(df_geo_high)))


Saved: counties_high_tier_majority.csv (rows: 0)


In [19]:
# ---------------------------
# 9) Wrap up & close connection
# ---------------------------
print("Done. Outputs written to '{}'.".format(OUT_DIR))
conn.close()

Done. Outputs written to 'formulary_outputs'.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
file_path = 'Programs\Features\formulary_outputs\tier_by_geo_state_county.csv'
df = pd.read_csv(file_path)

# Drop rows with missing tier_level_value for better visualization
df_clean = df.dropna(subset=['tier_level_value'])

# Convert tier_level_value to integer for plotting
df_clean['tier_level_value'] = df_clean['tier_level_value'].astype(int)

# Plot number of plans by tier level using a bar plot
plt.figure(figsize=(12, 6))
sns.barplot(x='tier_level_value', y='n_plans', data=df_clean, ci=None)
plt.title('Number of Plans by Tier Level')
plt.xlabel('Tier Level')
plt.ylabel('Number of Plans')
plt.show()


<>:6: SyntaxWarning: invalid escape sequence '\F'
<>:6: SyntaxWarning: invalid escape sequence '\F'
C:\Users\amsub\AppData\Local\Temp\ipykernel_53760\1841559526.py:6: SyntaxWarning: invalid escape sequence '\F'
  file_path = 'Programs\Features\formulary_outputs\tier_by_geo_state_county.csv'
C:\Users\amsub\AppData\Local\Temp\ipykernel_53760\1841559526.py:6: SyntaxWarning: invalid escape sequence '\F'
  file_path = 'Programs\Features\formulary_outputs\tier_by_geo_state_county.csv'


FileNotFoundError: [Errno 2] No such file or directory: 'Programs\\Features\x0cormulary_outputs\tier_by_geo_state_county.csv'